In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hsoub/L8/emails.csv',sep=';' )
df.head()

,Spam,Message
0,0,Please call me at 8
1,1,Free money is available for you
2,0,I study he studies they are students I studied...
3,1,I am working at office now to 9 evening
4,0,U dun say so early hor... U c already then say...


#

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import string
stop_words = stopwords.words('english')
stemmer = PorterStemmer()

In [ ]:
import re
def preprocess_text(text):

    tokens = word_tokenize(text.lower())


    tokens = [token for token in tokens if token not in string.punctuation]


    stop_words = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop_words]


    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]


    preprocessed_text = ' '.join(tokens)

    preprocessed_text = re.sub(r'http\S+|www\S+', '', preprocessed_text)

    preprocessed_text = re.sub(r'\d+', '', preprocessed_text)
    return preprocessed_text

In [ ]:
df['processed_Message']=df['Message'].apply(preprocess_text)
df

,Spam,Message,processed_Message
0,0,Please call me at 8,pleas call
1,1,Free money is available for you,free money avail
2,0,I study he studies they are students I studied...,studi studi student studi yesterday
3,1,I am working at office now to 9 evening,work offic even
4,0,U dun say so early hor... U c already then say...,u dun say earli hor ... u c alreadi say ...
...,...,...,...
5222,0,"It‘s reassuring, in this crazy world.",‘ reassur crazi world
5223,0,Oh... Okie lor...We go on sat...,oh ... oki lor ... go sat ...
5224,1,You are awarded a SiPix Digital Camera! call 0...,award sipix digit camera call landlin deliver...
5225,0,"Hey chief, can you give me a bell when you get...",hey chief give bell get need talk royal visit ...


In [ ]:
df[df['Spam']==1].count()

,0
Spam,675
Message,675
processed_Message,675


#

In [ ]:
df[df['Spam']==0].count()

,0
Spam,4552
Message,4552
processed_Message,4552


In [ ]:
spam_df = df[df['Spam'] == 1]
non_spam_df = df[df['Spam'] == 0]

max_class_count = len(non_spam_df)

spam_resampled = spam_df.sample(n=max_class_count, replace=True, random_state=42)

balanced_df = pd.concat([non_spam_df, spam_resampled])

balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
sentences=balanced_df['processed_Message']
labels=balanced_df['Spam']
len(sentences),len(labels)

(9104, 9104)

In [ ]:
#

In [ ]:
model_name = 'bert-base-uncased'

In [ ]:
from transformers import BertTokenizer
bert_tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
sent = "Don't like it!"
tokens = bert_tokenizer.tokenize(sent)
print(tokens)

['don', "'", 't', 'like', 'it', '!']


In [ ]:
max_seq_length = 10

In [ ]:
bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,
                                        max_length =max_seq_length,
                                        truncation=True,
                                        return_attention_mask = True)
input_ids=bert_inp['input_ids']
token_type_ids=bert_inp['token_type_ids']
attention_mask=bert_inp['attention_mask']
tokens= bert_tokenizer.decode(input_ids)

print("Tokens: ", tokens)
print("Tokens IDs: ", input_ids)
print("Tokens Types IDs: ", token_type_ids)
print("Attention Mask: ", attention_mask)

Tokens:  [CLS] don't like it! [SEP]
Tokens IDs:  [101, 2123, 1005, 1056, 2066, 2009, 999, 102]
Tokens Types IDs:  [0, 0, 0, 0, 0, 0, 0, 0]
Attention Mask:  [1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
from keras.preprocessing.sequence import pad_sequences

input_ids = pad_sequences([input_ids], maxlen=max_seq_length, padding='post')
token_type_ids= pad_sequences([token_type_ids], maxlen=max_seq_length, padding='post')
attention_mask = pad_sequences([attention_mask], maxlen=max_seq_length, padding='post')

In [ ]:
print("Tokens: ", bert_tokenizer.decode(input_ids[0]))
print("Tokens IDs: ", input_ids[0])
print("Tokens Types IDs: ", token_type_ids)
print("Attention Mask: ", attention_mask)

Tokens:  [CLS] don't like it! [SEP] [PAD] [PAD]
Tokens IDs:  [ 101 2123 1005 1056 2066 2009  999  102    0    0]
Tokens Types IDs:  [[0 0 0 0 0 0 0 0 0 0]]
Attention Mask:  [[1 1 1 1 1 1 1 1 0 0]]


In [ ]:
max_seq_length = 32

In [ ]:
input_ids = []
attention_masks = []
for sent in sentences:
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,
                                        max_length =max_seq_length,
                                        truncation=True,
                                        return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])


In [ ]:
sentences[11]

'how pain dear r u smile'

In [ ]:
input_ids[11]

[101, 2129, 3255, 6203, 1054, 1057, 2868, 102]

In [ ]:
attention_masks[11]

[1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
from keras.preprocessing.sequence import pad_sequences
input_ids = pad_sequences(input_ids, maxlen=max_seq_length, padding='post')
attention_masks = pad_sequences(attention_masks, maxlen=max_seq_length, padding='post')


In [ ]:
input_ids[11]

array([ 101, 2129, 3255, 6203, 1054, 1057, 2868,  102,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [ ]:
input_ids=np.array(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

In [ ]:
#

In [ ]:
len(input_ids),len(attention_masks),len(labels)

(9104, 9104, 9104)

In [ ]:
#

In [ ]:
from transformers import TFBertForSequenceClassification
num_classes=2

model_name = 'bert-base-uncased'

bert_model = TFBertForSequenceClassification.from_pretrained(model_name,
                                                             num_labels=num_classes)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bert_model.summary()

Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_187 (Dropout)       multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import tensorflow as tf
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

In [ ]:
bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metric])
bert_model.summary()

Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_187 (Dropout)       multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from sklearn.model_selection import train_test_split
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split (input_ids,labels, attention_masks,test_size =0.2)

In [ ]:
epochs=4
batch_size=32
history=bert_model.fit([train_inp,train_mask],
                       train_label,batch_size=batch_size,epochs=epochs,
                       validation_data=([val_inp,val_mask],val_label))

Epoch 1/4


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
228/228 [==============================] - 418s 2s/step - loss: 0.1439 - accuracy: 0.9488 - val_loss: 0.0451 - val_accuracy: 0.9863
Epoch 2/4
228/228 [==============================] - 323s 1s/step - loss: 0.0187 - accuracy: 0.9946 - val_loss: 0.0177 - val_accuracy: 0.9962
Epoch 3/4
228/228 [==============================] - 324s 1s/step - loss: 0.0091 - accuracy: 0.9971 - val_loss: 0.0729 - val_accuracy: 0.9852
Epoch 4/4
228/228 [==============================] - 328s 1s/step - loss: 0.0067 - accuracy: 0.9978 - val_loss: 0.0273 - val_accuracy: 0.9934


In [ ]:
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
print("Train Accuracy:", round(100*train_accuracy[-1],2))
print("Validation Accuracy:", round(100*val_accuracy[-1],2))

Train Accuracy: 99.78
Validation Accuracy: 99.34


In [ ]:
model_save_path='/content/drive/MyDrive/Colab Notebooks/Hsoub/L8full_ft_bert_model.h5'
bert_model.save_weights(model_save_path)

In [ ]:
trained_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
trained_model.load_weights(model_save_path)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.metrics import classification_report
target_names=['Not Spam','Spam']
preds = trained_model.predict([val_inp,val_mask],batch_size=32)
pred_labels = preds.logits.argmax(axis=1)

57/57 [==============================] - 31s 390ms/step


In [ ]:
print('Classification Report')
print(classification_report(val_label,pred_labels,target_names=target_names))

Classification Report
              precision    recall  f1-score   support

    Not Spam       1.00      0.99      0.99       910
        Spam       0.99      1.00      0.99       911

    accuracy                           0.99      1821
   macro avg       0.99      0.99      0.99      1821
weighted avg       0.99      0.99      0.99      1821



In [ ]:
sent="free money"
input_ids_sent=[]
attention_masks_sent=[]

bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,
                                    max_length =max_seq_length,
                                    truncation=True,
                                    return_attention_mask = True)
input_ids_sent.append(bert_inp['input_ids'])
attention_masks_sent.append(bert_inp['attention_mask'])
input_ids_sent = pad_sequences(input_ids_sent, maxlen=max_seq_length, padding='post')
attention_masks_sent = pad_sequences(attention_masks_sent, maxlen=max_seq_length, padding='post')
input_ids_sent=np.array(input_ids_sent)
attention_masks_sent=np.array(attention_masks_sent)

In [ ]:
predictions =trained_model.predict([input_ids_sent,attention_masks_sent])
print(predictions)


1/1 [==============================] - 0s 88ms/step
TFSequenceClassifierOutput(loss=None, logits=array([[-3.5257175,  2.9950688]], dtype=float32), hidden_states=None, attentions=None)


In [ ]:
c=np.argmax(predictions.logits[0])
if c==0:
    print("The text is predicted to be of class: Not Spam")
else:
    print("The text is predicted to be of class: Spam")

The text is predicted to be of class: Spam
